In [7]:
import pandas as pd
import csv
import warnings
warnings.filterwarnings("ignore")

In [8]:
# Função para ler o csv
# O CSV deve estar com este nome: 'FILA.csv', ou deve ser alterado na função
# O arquivo deve estar no mesmo repositório do script, ou deve ser especificado na função
ura = pd.read_csv('FILA.csv', sep =";",dtype = str)

In [9]:
# Verificando o número de linahs
nrows = ura.shape[0]
nrows

1278

In [10]:
# Visualizando o dataframe gerado
ura

,caracteres,CustomerID,NumeroPedido,items_id,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,14,336.645.408-32,711131352,14912509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14,470.106.438-65,711197352,14886860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14,410.617.718-81,711232352,14854304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14,305.183.098-43,711376152,14778018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,285.669.418-70,711382152,14768018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273,14,336.933.258-20,799847352,14883305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1274,14,350.119.608-38,799859412,14957692,14957693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1275,14,223.418.888-10,799952412,14938266,14938268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1276,14,415.500.688-06,799957352,14889964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Função que remove os CNPJs seguida por uma função que reorganiza os índices para evitar erros
ura = ura[ura['caracteres'] == '14']
ura = ura.reset_index()

In [12]:
# Criando uma lista para poder acessar os itens por índices
colunas = []
for i in ura.columns:
    colunas.append(i)
colunas[5]

'Unnamed: 4'

In [13]:
# Função que lê linha a linha do dataframe em busca de valores não nulos
# Ao encontrar um valor nulo, ela cria uma nova linha colocando este valor como item_id e copiando o cpf e numero de pedido
# da linha na qual ele foi encontrado
# foi utilizada a função TRY para que o programe rode uma "correção" para erros gerados por valores não esperados e não nulos
for i in range(ura.shape[0]):
        
        for j in range(5,len(colunas)):
            
            if pd.isna(ura[colunas[j]][i]) == False:
                
                df_inserido = pd.DataFrame([[ura[colunas[2]][i],ura[colunas[3]][i],ura[colunas[j]][i]]],columns=['CustomerID', 'NumeroPedido','items_id'])
                ura = pd.concat([ura, df_inserido])
                ura[colunas[j]][i] = None
                ura = ura.reset_index()
                ura = ura.drop(['level_0'], axis = 1)

In [14]:
ura.shape[0]

1500

In [15]:
for i in ura.columns:
    if i not in ['CustomerID','NumeroPedido','items_id']:
        ura = ura.drop(i,axis=1)

In [16]:
ura['items_id'] = ura['items_id'].astype(int)
ura['items_id'] = ura['items_id'].astype(str)
ura['CustomerID'] = ura['CustomerID'].str.replace(".","")
ura['CustomerID'] = ura['CustomerID'].str.replace("-","")
ura['NumeroPedido'] = ura['NumeroPedido'].astype(int)
ura['NumeroPedido'] = ura['NumeroPedido'].astype(str)

In [17]:
ura = ura.rename({'CustomerID': "clientDocument", 'NumeroPedido': 'orderCod' , 'items_id': 'orderItemCod' }, axis=1)

In [18]:
ura

,clientDocument,orderCod,orderItemCod
0,33664540832,711131352,14912509
1,47010643865,711197352,14886860
2,41061771881,711232352,14854304
3,30518309843,711376152,14778018
4,28566941870,711382152,14768018
...,...,...,...
1495,23033718809,797699412,14954570
1496,23033718809,797699412,14954571
1497,08616964843,799256412,14948317
1498,35011960838,799859412,14957693


In [19]:
# Cria CSVs com no máx 250 linhas, eles serão salvos no mesmo diretório do script
j = ura.shape[0] // 250 + 1
for i in range(1,j+1):
    if i == j:
        m = (i-1)*250
        n = (ura.shape[0] % 250)+m
        file_name = "FILA_URA"+ str(i)+".xlsx"
        aux = ura[m:n][:]
        aux.to_excel(file_name, index = False)
    else:
        file_name = "FILA_URA"+ str(i) +".xlsx"
        m = (i-1)*250
        n = i*250
        aux = ura[m:n][:]
        aux.to_excel(file_name, index = False)